In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
from qiskit.visualization import plot_histogram
from typing import List

import numpy as np

# Find the Largest Number
## QRAM

Before designing the quantum algorithm that gives the largest number, we have to define how the data in the list can be accessed quantumly. We want a circuit $U_f$ such that:

$$
\ket{i}\ket{0} \xrightarrow{U_f} \ket{i}\ket{f(i)}
$$

where $i$ is the index and $f(i)$ gives the ith number in the list.

For each integer in the list, we choose basic/state encoding, the highest bit is sign bit. For example, 5 and -6 can be encoded as:

$$\begin{align*}
5  &\rightarrow \ket{00101} \\
-6 &\rightarrow \ket{11010}      
\end{align*}$$

4 bits is enough to hold 5 and -6. Since we'll later do addition and subtraction between two numbers, we use 1 extra bit to prevent overflow. The indeices will also be encoded using state encoding and we encode the them into the ctrl bits of mcx gate. The control bit will be controlled by either '0' or '1' based on the state encoding of the index on that position.

In [2]:
"""
Here is a general method to convert a list of number into quantum data.

In this function, we used np.max to find the bound for the list, 
but such information is not encoded into the QRAM so the algorithm that we'll 
present in a moment to solve the problem will not lost any generality. And in practice 
the bound on integer is always predetermined by its type like 'int' in C usually represents 
integers from -2147483648 to 2147483647. The reason that we choose to dynamically determine 
the number of qubit is for compactness and clarity.

"""

def toQRAM(
    data_list: List[int]
) -> QuantumCircuit:
    """
        returns the circuit U_f such that U_f|i>|0> = |i>|x_i>
    """
    
    # define minimum number of qubit to represent all the integers in the list
    bound = np.max(np.abs(data_list))
    num_qubit_i = int(np.ceil(np.log2(len(data_list))))
    num_qubit_xi = len(bin(bound)[2:]) + 2 # one for sign and one for preventing overflow
    
    # define index and data quantum registers
    q_i  = QuantumRegister(num_qubit_i, "q_i")
    q_xi = QuantumRegister(num_qubit_xi, "q_xi")

    c_i = ClassicalRegister(num_qubit_i, "i")
    c_xi = ClassicalRegister(num_qubit_xi, "xi")
    
    qc = QuantumCircuit(q_i, q_xi, c_i, c_xi)

    for i, x in enumerate(data_list):
        # convert index into binary and agree with the number of index qubit
        i_bin = format(i, f'0{num_qubit_i}b') 
        # convert x_i into binary and agree with the number of data qubit
        x_bin = bin(x & int('1' * num_qubit_xi, 2))[2:]

        # determine which control bit is controlled by '0'
        for i_bin_idx, i_bin_symbol in enumerate(reversed(i_bin)):
            if '0' == i_bin_symbol:
                qc.x(q_i[i_bin_idx])
        
        # encode x_i
        for x_bin_idx, x_bin_symbol in enumerate(reversed(x_bin)):
            if '1' == x_bin_symbol:
                qc.mcx(q_i, q_xi[x_bin_idx])
        
        # uncompute for the control bit
        for i_bin_idx, i_bin_symbol in enumerate(reversed(i_bin)):
            if '0' == i_bin_symbol:
                qc.x(q_i[i_bin_idx])

    return qc

For example the list [-3,2,5,-1] in the qram form will be:

In [3]:
ex_list = [-3, 2, 5, 1]

qram_circ = toQRAM(ex_list)
qram_circ.draw()

┌───┐                    ┌───┐          ┌───┐          ┌───┐     
 q_i_0: ┤ X ├──■────■────■────■──┤ X ├───────■──┤ X ├──■────■──┤ X ├──■──
        ├───┤  │    │    │    │  ├───┤┌───┐  │  ├───┤  │    │  └───┘  │  
 q_i_1: ┤ X ├──■────■────■────■──┤ X ├┤ X ├──■──┤ X ├──■────■─────────■──
        └───┘┌─┴─┐  │    │    │  └───┘└───┘  │  └───┘┌─┴─┐  │       ┌─┴─┐
q_xi_0: ─────┤ X ├──┼────┼────┼──────────────┼───────┤ X ├──┼───────┤ X ├
             └───┘  │    │    │            ┌─┴─┐     └───┘  │       └───┘
q_xi_1: ────────────┼────┼────┼────────────┤ X ├────────────┼────────────
                  ┌─┴─┐  │    │            └───┘          ┌─┴─┐          
q_xi_2: ──────────┤ X ├──┼────┼───────────────────────────┤ X ├──────────
                  └───┘┌─┴─┐  │                           └───┘          
q_xi_3: ───────────────┤ X ├──┼──────────────────────────────────────────
                       └───┘┌─┴─┐                                        
q_xi_4: ────────────────────┤ X ├────────────────────────────────────────
                            └───┘                                        
   i: 2/═════════════════════════════════════════════════════════════════
                                                                         
  xi: 5/═════════════════════════════════════════════════════════════════

And we can confirm the qram indeed stores the disered data by making query on all possible indices:

In [4]:
query_circ = QuantumCircuit(qram_circ.qregs[0])
# equal supperposition on all indices
query_circ.h(query_circ.qregs[0])

query_circ.barrier()
# add the query in fron of the qram
query_circ = qram_circ.compose(query_circ, query_circ.qregs[0], front=True)
query_circ.barrier()

# measure and get results of the queries
query_circ.measure(query_circ.qregs[0], query_circ.cregs[0])
query_circ.measure(query_circ.qregs[1], query_circ.cregs[1])

#query_circ.draw()

In [5]:
# run the circuit to get the result of the queries
simulator = Aer.get_backend('qasm_simulator')
result = execute(query_circ, simulator).result()
counts = result.get_counts()
# raw output in binary
print(counts)

# extract original classical data 
for xi_i in counts:
    xi_bin, i_bin = xi_i.split(" ")
    
    # have to consider the sign when convert back into integer 
    sign_bit = xi_bin[0]
    xi = int(xi_bin[1:], 2)
    if '1' == sign_bit:
        xi = -(2**(len(xi_bin)-1) - xi)
    # index is unsigned
    i = int(i_bin, 2)

    print(f"f[{i}]: {xi}")

{'00101 10': 241, '00001 11': 272, '00010 01': 262, '11101 00': 249}
f[2]: 5
f[3]: 1
f[1]: 2
f[0]: -3


We can see the recovered data is exactly the same as the original list

## Quantum-classical Adder

With classically given integer X, positive or negative, we want the adder to do:

$$
    \ket{n} \xrightarrow{} \ket{n + X}
$$

